## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Leua,SK,48.2156,18.6071,68.99,64,99,4.88,overcast clouds
1,1,Barrow,US,71.2906,-156.7887,35.62,93,100,32.21,overcast clouds
2,2,Padang,ID,-0.9492,100.3543,84.25,73,99,3.24,overcast clouds
3,3,Nalut,LY,30.3333,10.8500,78.31,24,0,7.23,clear sky
4,4,Ribeira Grande,PT,38.5167,-28.7000,74.39,87,13,6.71,few clouds
...,...,...,...,...,...,...,...,...,...,...
693,693,Saint-Francois,FR,46.4154,3.9054,60.67,82,75,5.75,broken clouds
694,694,Sukhobezvodnoye,RU,57.0491,44.8891,75.27,64,15,6.89,few clouds
695,695,Socorro,US,31.6546,-106.3033,82.26,57,75,16.11,thunderstorm
696,696,Nara,JP,34.6851,135.8049,94.96,52,97,9.08,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 79


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(how="any", inplace=True)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Nalut,LY,78.31,clear sky,30.3333,10.8500,
6,Westport,US,76.48,overcast clouds,41.1415,-73.3579,
12,Rikitea,PF,75.09,broken clouds,-23.1203,-134.9692,
15,Ilam,NP,75.04,overcast clouds,26.9167,87.9167,
26,Port-Gentil,GA,75.16,overcast clouds,-0.7193,8.7815,
29,Kilindoni,TZ,76.84,scattered clouds,-7.9139,39.6668,
37,Takestan,IR,78.93,few clouds,36.0696,49.6959,
52,Hattiesburg,US,78.73,clear sky,31.3271,-89.2903,
54,Pahrump,US,78.31,overcast clouds,36.2083,-115.9839,
58,Nouadhibou,MR,76.98,broken clouds,20.9310,-17.0347,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")
clean_hotel_df.count()

City                   87
Country                87
Max Temp               87
Current Description    87
Lat                    87
Lng                    87
Hotel Name             87
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather_Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))